In [1]:
!pip install pandas numpy scikit-learn


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
new_df = pd.read_csv('tmdb.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'tmdb.csv'

In [4]:
# ----------------------------------------------
# 🎬 MOVIE RECOMMENDATION SYSTEM USING TMDB DATASET
# ----------------------------------------------

# STEP 1: Import Libraries
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# STEP 2: Load Dataset
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

# STEP 3: Merge both datasets on 'title'
movies = movies.merge(credits, on='title')

# STEP 4: Select important features
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

# STEP 5: Handle missing data
movies.dropna(inplace=True)

# STEP 6: Convert JSON-like strings to Python objects
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

# STEP 7: Extract top 3 cast members
def convert_cast(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count < 3:
            L.append(i['name'])
            count += 1
    return L

movies['cast'] = movies['cast'].apply(convert_cast)

# STEP 8: Extract director’s name
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

movies['crew'] = movies['crew'].apply(fetch_director)

# STEP 9: Convert overview into list of words
movies['overview'] = movies['overview'].apply(lambda x: x.split())

# STEP 10: Remove spaces between multi-word names
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

# STEP 11: Combine all into a single tag column
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

# STEP 12: New dataframe with useful columns
new_df = movies[['movie_id','title','tags']]

# STEP 13: Convert tags list to string
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

# STEP 14: Convert all to lowercase
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

# STEP 15: Text Vectorization
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

# STEP 16: Compute similarity using cosine similarity
similarity = cosine_similarity(vectors)

# STEP 17: Define recommendation function
def recommend(movie):
    movie = movie.lower()
    if movie not in new_df['title'].str.lower().values:
        print("❌ Movie not found. Please check spelling.")
        return
    index = new_df[new_df['title'].str.lower() == movie].index[0]
    distances = similarity[index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    print(f"\n🎥 Top 5 movies similar to '{new_df.iloc[index].title}':\n")
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

# ----------------------------------------------
# ✅ TEST THE SYSTEM
# ----------------------------------------------

# Example:
recommend('avatar')    # You can change movie name here


C:\Users\thaku\AppData\Local\Temp\ipykernel_9600\1452129093.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
C:\Users\thaku\AppData\Local\Temp\ipykernel_9600\1452129093.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())



🎥 Top 5 movies similar to 'Avatar':

Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem


In [5]:
recommend('Spectre')


🎥 Top 5 movies similar to 'Spectre':

Quantum of Solace
Never Say Never Again
Skyfall
From Russia with Love
Thunderball


In [6]:
recommend('Spider-Man 3')


🎥 Top 5 movies similar to 'Spider-Man 3':

Spider-Man 2
Spider-Man
The Amazing Spider-Man 2
The Amazing Spider-Man
Arachnophobia
